In [1]:
:opt no-lint
-- Required language extensions
{-# LANGUAGE DataKinds #-}             -- typelevel lists and tuples
{-# LANGUAGE GADTs #-}                 -- task declaration
{-# LANGUAGE OverloadedLabels #-}      -- named task disambiguation
{-# LANGUAGE OverloadedStrings #-}     -- flexible string types
{-# LANGUAGE RankNTypes #-}            -- polymorphism of Flow

-- core types and functions used here
import Funflow.Config (render, Configurable (..), ExternalConfig)

-- other necessary imports
import System.Environment (setEnv, unsetEnv)

# Custom tasks, `Configurable`s, and type checking

In this tutorial, we'll show how to build a `Flow` that uses as custom task type.
Additionally, we'll demonstrate `funflow`'s type safety for `Configurable` parameters.

## Defining a custom task type and its _interpreter_

Here, we declare a custom task type and then define __what it does__ with an _interpreter_ function.

In [2]:
import Control.Arrow (Arrow, arr)

-- Define the representation of a custom task with some String and Int parameters.
-- We'll use this as a sort of closure, yielding a "function" (task) with text input and output.
-- This is just the declaration; the closure-like logic lies in the implementation of this task's interpreter.
data EchoXTask i o where
    EchoXTask :: String -> Configurable Int -> EchoXTask String String

-- our custom task's interpreter; essentially, this is what the task does.
interpretEcho :: (Arrow a) => ExternalConfig -> EchoXTask i o -> a i o
interpretEcho extCfg (EchoXTask msg x@(ConfigFromEnv k)) = 
    case render x extCfg of Left msg           -> error msg
                            Right (Literal n)  -> arr (++ repStr (n,msg))
                            Right _            -> error $ "Unrendered env-based config: " ++ show k
        where repStr = concat . uncurry replicate
-- Here we choose to demo just environment-based configurability, but the same applies for files.
interpretEcho _ _ = error "This implementation only supports environment-based configuration."

## Tell `funflow` about our task.

With our custom task type declared and accompanied by an interpreter, which defines what it should do, we now must tell `funflow` about it so that a `Flow` that uses this task type can be executed.

First we create a flow type that includes our task type.

In [3]:
import Funflow (ExtendedFlow)

-- Create a flow type that knows about (and can run) our custom task type.
type EchoFlow i o = ExtendedFlow '[ '("echodemo", EchoXTask) ]  i o

Now we provide a convenience function, or a "smart constructor," to facilitate create values of this custom flow type.

In [4]:
import Control.Kernmantle.Rope (strand)

buildEchoX :: String -> Configurable Int -> EchoFlow String String
buildEchoX refrain nRepsParam = strand #echodemo (EchoXTask refrain nRepsParam)

Finally, we "weave" the "strand" comprised of our custom task type, its label, and its interpreter, into the overall "rope" that constitutes the flow. For more on this, [refer to `kernmantle`](https://github.com/tweag/kernmantle).

In [5]:
import Control.Kernmantle.Rope ((&), weave')

weaveMyFlow myFlow myConfig = myFlow & weave' #echodemo (interpretEcho myConfig)

## Running the flow with our custom task

Here we provide the functions to execute our custom flow.

In [6]:
:opt no-lint
import Funflow.Run (runFlow)

-- Use a configuration, a flow value, and an input value to generate an output.
runMyFlow :: ExternalConfig -> EchoFlow i o -> i -> IO o
runMyFlow config myFlow input = runFlow (weaveMyFlow myFlow config) input

In [7]:
import Funflow.Config (readEnv, ExternalConfig (..), fileConfig, envConfig)
import qualified Data.Text as Text

-- Accept the name of the env var that provides the repetition count for the echo task.
execDemo :: String -> IO String
execDemo envVarRaw = 
    let envVar = Text.pack envVarRaw
    in readEnv envVar >>= (\envMap -> 
        let envCfg = ExternalConfig{ fileConfig = mempty, envConfig = envMap }
        in runMyFlow envCfg (buildEchoX "Refrain" (ConfigFromEnv envVar)) "I'm the PREFIX: ")

## Configurable parameters must be defined and correctly typed.

In [36]:
import Control.Exception (throw, try, SomeException)

-- env var we'll use and reuse.
myVarName :: String
myVarName = "DEMO_TEMP"

runSafe :: String -> IO ()
runSafe varname = do
    result <- try (execDemo varname) :: IO (Either SomeException String)
    case result of Left ex -> putStrLn ("ERROR! " ++ show ex)
                   Right msg -> putStrLn ("Non-error result: " ++ msg)

First, as expected, when the environment variable that we're using to parameterize our run attempt isn't set, the flow fails.

In [37]:
unsetEnv myVarName
runSafe myVarName

ERROR! Failed to extract configurable DEMO_TEMP from environment variable with error: Failed to find key 'DEMO_TEMP' in provided config.
CallStack (from HasCallStack):
  error, called at <interactive>:5:51 in interactive:Ghci20

Not only must a configurable parameter be set, but it __also must type check__.
In the case of an environment variable, for example, this is trivial if the `Configurable` type is `String`. 
If we declare a configurable as some other type, though, then pre-execution "interpretation" of a task provides the desirable _fail fast_ behavior. 

In [38]:
setEnv myVarName "wrong type!"
runSafe myVarName

ERROR! Failed to extract configurable DEMO_TEMP from environment variable with error: Aeson exception:
Error in $: parsing Int failed, expected Number, but encountered String
CallStack (from HasCallStack):
  error, called at <interactive>:5:51 in interactive:Ghci20

While here our "pipeline" is trivial, in general this upfront type safety saves us the wasted time and resources of running something that we can deduced is doomed to crash.

Finally, when we set the config key's value to something parseable as intended, the flow succeeds.

In [39]:
setEnv myVarName "3"
runSafe myVarName

Non-error result: I'm the PREFIX: RefrainRefrainRefrain

Note that the value parsing uses `Data.Yaml`; a custom type may be used for a `Configurable` as long as the type has a `FromJSON` instance available.